$Setting~Up~Environment$

In [1]:
import pandas as pd
from gensim.models.phrases import Phrases, Phraser
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import joblib

$Load~in~dataset$

In [28]:
misinfo_covid = '/Users/alexz/Downloads/covid_misinfo_unesco/disinfo_stories.csv'
misinfo_covid_df = pd.read_csv(misinfo_covid)
title = misinfo_covid_df['Title']

$Extract~phrases/sentences~from~title~column~in~dataset$

In [37]:
# Check for NaN values in 'title' and replace them with an empty string (otherwise rows won't split, will show error)
title = title.fillna('')
# Split each row in 'title' only if it is a string
sent = [row.split() if isinstance(row, str) else [] for row in title]
#Create phrases object from Phrases method in gensim library
phrases = Phrases(sent, min_count=30, progress_per=10000)
#Create a Phraser object called sentences which contains sentences inside from the original dataframe column "title"
sentences = phrases[sent]

NameError: name 'gensim' is not defined

$Initializing,~building~ vocabulary,~and~training~for~fastText~model$

In [35]:
#Build FastText vocabulary
model_covid_misinfo = FastText(vector_size=100, window=5, min_count=5, workers=4, min_n=1) #size=n has been deprecated
model_covid_misinfo.build_vocab(sentences)
print(len(model_covid_misinfo.wv.key_to_index)) 

1640


In [38]:
#Train FastText model
model_covid_misinfo.train(sentences, total_examples=len(sentences), epochs=100)

(2933101, 6190900)

In [40]:
#Save model for later
path = 'fasttext_covidmisinfo.joblib'
joblib.dump(model_covid_misinfo,path)

['fasttext_covidmisinfo.joblib']

$Looking~ at~ 'Most~ Similar'~ word~ ratings: FastText$

In [87]:
#In vocabulary
model_covid_misinfo.wv.most_similar('government',topn=10)
model_covid_misinfo.wv.most_similar('response',topn=10)
model_covid_misinfo.wv.most_similar('vaccine',topn=10)
model_covid_misinfo.wv.most_similar('hydroxychloroquine',topn=10)
model_covid_misinfo.wv.most_similar('good',topn=10)


ngram_buckets=model_covid_misinfo.wv.recalc_char_ngram_buckets()
ngram_buckets
model_covid_misinfo.wv.most_similar('government',topn=10)
model_covid_misinfo.wv.n_similarity(['government'],['response'])

#model_covid_misinfo.wv.rank_by_centrality()
#rank(self, key1, key2)
#help(model_covid_misinfo.wv)
#n_similarity(self, ws1, ws2)
#most_similar_cosmul(self, positive=None, negative=None, topn=10, restrict_vocab=None)
#most_similar_to_given(self, key1, keys_list)

0.40615907 0.30257067 0.361479 0.59322786


In [110]:
def calculate_similarities(model, pairs):
    results = []
    for word1, word2 in pairs:
        try:
            similarity = model.wv.n_similarity([word1], [word2])
            results.append({'word1': word1, 'word2': word2, 'similarity': similarity})
        except KeyError as e:
            results.append({'word1': word1, 'word2': word2, 'similarity': 'Word not in vocabulary'})
    return pd.DataFrame(results)

if __name__ == '__main__':
    model = model_covid_misinfo  # Load your trained model
    word_pairs = [('vaccine', 'bad'), ('vaccine', 'good'),
                  ('vaccine',  'effective'), ('vaccine', 'dangerous'),
                  ('vaccine', 'ineffective'), ('vaccine', 'deadly'),
                  ('government', 'good'), ('government', 'bad'),
                  ('response', 'good'), ('response', 'bad'),
                  ('hydroxychloroquine', 'good'), ('hydroxychloroquine', 'bad'),
                  ('America', 'good'), ('America', 'bad'),
                  ('China', 'good'), ('China', 'bad'),
                  ('Russia', 'good'), ('Russia', 'bad'),
                  ('India', 'good'), ('India', 'bad'),
                  ('China', 'fast'), ('China', 'slow'),
                  ('America', 'fast'), ('America', 'slow'),
                  ('Russia', 'fast'), ('Russia', 'slow'),
                  ('India', 'fast'), ('India', 'slow'),
                  ('government', 'fast'), ('government', 'slow'),
                  ('response', 'fast'), ('response', 'slow')
                  
                                        ]
    similarity_df = calculate_similarities(model, word_pairs)
    print(similarity_df)    

                 word1        word2  similarity
0              vaccine          bad    0.247462
1              vaccine         good    0.302571
2              vaccine    effective    0.708363
3              vaccine    dangerous    0.398112
4              vaccine  ineffective    0.759837
5              vaccine       deadly    0.426567
6           government         good    0.556639
7           government          bad    0.361479
8             response         good    0.497543
9             response          bad    0.352768
10  hydroxychloroquine         good    0.558569
11  hydroxychloroquine          bad    0.445593
12             America         good    0.396039
13             America          bad    0.437726
14               China         good    0.365770
15               China          bad    0.360919
16              Russia         good    0.394316
17              Russia          bad    0.474688
18               India         good    0.520436
19               India          bad    0

In [124]:
import matplotlib.pyplot as plt

def calculate_similarities(model, pairs):
    results = []
    for word1, word2 in pairs:
        try:
            similarity = model.wv.n_similarity([word1], [word2])
            results.append({'word1': word1, 'word2': word2, 'similarity': similarity})
        except KeyError as e:
            results.append({'word1': word1, 'word2': word2, 'similarity': 'Word not in vocabulary'})
    return pd.DataFrame(results)

if __name__ == '__main__':
    model = model_covid_misinfo  # Load your trained model
    word_pairs = [('vaccine', 'bad'), ('vaccine', 'good'),
                  ('vaccine',  'effective'), ('vaccine', 'dangerous'),
                  ('vaccine', 'ineffective'), ('vaccine', 'deadly'),
                  ('government', 'good'), ('government', 'bad'),
                  ('response', 'good'), ('response', 'bad'),
                  ('hydroxychloroquine', 'good'), ('hydroxychloroquine', 'bad'),
                  ('America', 'good'), ('America', 'bad'),
                  ('China', 'good'), ('China', 'bad'),
                  ('Russia', 'good'), ('Russia', 'bad'),
                  ('India', 'good'), ('India', 'bad'),
                  ('China', 'fast'), ('China', 'slow'),
                  ('America', 'fast'), ('America', 'slow'),
                  ('Russia', 'fast'), ('Russia', 'slow'),
                  ('India', 'fast'), ('India', 'slow'),
                  ('government', 'fast'), ('government', 'slow'),
                  ('response', 'fast'), ('response', 'slow')
                  
                                        ]
    similarity_df = calculate_similarities(model, word_pairs)
    print(similarity_df)    
import matplotlib.pyplot as plt

def plot_similarity_frequency(model, word, countries):
    frequencies = []
    
    for country in countries:
        similarity = model.wv.n_similarity([country], [word])
        similarity.append(similarity)
        frequency = model.wv.vocab.get(f"{country} - {word}", {}).count
        frequencies.append(frequency)
    
    plt.figure(figsize=(10, 6))
    plt.bar(countries, similarities, label='Similarity')
    plt.bar(countries, frequencies, label='Frequency')
    plt.xlabel('Country')
    plt.ylabel('Value')
    plt.title(f'Similarity and Frequency for "{word}"')
    plt.legend()
    plt.xticks(rotation=90)
    plt.show()
    
# Example usage
model = model_covid_misinfo  # Load your trained model
countries = ['India', 'China', 'United States', 'Brazil', 'Russia']
plot_similarity_frequency(model_covid_misinfo, 'good', countries)
    


                 word1        word2  similarity
0              vaccine          bad    0.247462
1              vaccine         good    0.302571
2              vaccine    effective    0.708363
3              vaccine    dangerous    0.398112
4              vaccine  ineffective    0.759837
5              vaccine       deadly    0.426567
6           government         good    0.556639
7           government          bad    0.361479
8             response         good    0.497543
9             response          bad    0.352768
10  hydroxychloroquine         good    0.558569
11  hydroxychloroquine          bad    0.445593
12             America         good    0.396039
13             America          bad    0.437726
14               China         good    0.365770
15               China          bad    0.360919
16              Russia         good    0.394316
17              Russia          bad    0.474688
18               India         good    0.520436
19               India          bad    0

AttributeError: 'numpy.float32' object has no attribute 'append'